In [1]:
import numpy as np
import pandas as pd
import sys
from bs4 import BeautifulSoup as BSoup
import requests

In [2]:
formLink = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247"
showLink = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247"
r = requests.get(formLink)
soup = BSoup(r.text, "lxml")
selectFields = soup.find_all("select")
infoField=selectFields[0].find("option", text="Informatique")["value"]
infoFieldParam=selectFields[0]["name"]
bsSem1=selectFields[2].find("option", text="Bachelor semestre 1")["value"]
bsSem6=selectFields[2].find("option", text="Bachelor semestre 6")["value"]
semParam=selectFields[2]["name"]
allYears = [y["value"] for y in selectFields[1].find_all("option")[1:]]
yearParam = selectFields[1]["name"]
htmlradiobutton=soup.find("input", type="radio")

In [3]:
def showLinkGen(htmlP, htmlV, fieldP, fieldV, yearP, yearV, semP, semV):
  return "{}&{}={}&{}={}&{}={}&{}={}".format(showLink, htmlP, htmlV, fieldP, fieldV, yearP, yearV, semP, semV)
def get_bs_html(bsSem, year):
    link = showLinkGen(htmlradiobutton["name"], htmlradiobutton["value"], infoFieldParam, infoField, yearParam, 
                 year, semParam, bsSem)
    return requests.get(link)
def get_bs_dataframe(request_link):
  soup2 = BSoup(request_link.text, "lxml")
  elems=soup2.find_all("tr")[2:]
  titleinfo = soup2.find("font").text.split(', ')
  semester=titleinfo[2]
  semester=int(semester[(len(semester)-2):])
  year=titleinfo[1]
  all_data=[]
  for elem in elems:
    items=elem.find_all("td")
    gender = "M" if (items[0].text == "Monsieur") else "F"
    sciper = int(items[10].text)
    all_data.append({"Scipper": sciper, "Sex": gender, "Year": year, "Semester": semester})
  return pd.DataFrame(all_data)

In [4]:
def get_bs_alldata():
  all_data = []
  for bsSem in [bsSem1, bsSem6]:
    for year in allYears:
        all_data.append(get_bs_dataframe(get_bs_html(bsSem, year)))
  return pd.concat(all_data)
bachelor_data = get_bs_alldata()

In [5]:
bachelor_data

,Scipper,Semester,Sex,Year
0,235688,1,M,2016-2017
1,274015,1,M,2016-2017
2,268410,1,F,2016-2017
3,271464,1,M,2016-2017
4,274518,1,M,2016-2017
5,249613,1,M,2016-2017
6,262214,1,M,2016-2017
7,262239,1,M,2016-2017
8,257916,1,M,2016-2017
9,271508,1,F,2016-2017


In [6]:
data = bachelor_data.copy()

In [7]:
data.reset_index(None,drop=True,inplace=True)
data

,Scipper,Semester,Sex,Year
0,235688,1,M,2016-2017
1,274015,1,M,2016-2017
2,268410,1,F,2016-2017
3,271464,1,M,2016-2017
4,274518,1,M,2016-2017
5,249613,1,M,2016-2017
6,262214,1,M,2016-2017
7,262239,1,M,2016-2017
8,257916,1,M,2016-2017
9,271508,1,F,2016-2017


In [8]:
for i in range(data.shape[0]):
    if (data.loc[i,'Semester'] == 1):
        data.loc[i,'Year'] = data.loc[i,'Year'][0:4]
    else:
        data.loc[i,'Year'] = data.loc[i,'Year'][5:9]

data['Year'] = data['Year'].astype('int')
data.head()

,Scipper,Semester,Sex,Year
0,235688,1,M,2016
1,274015,1,M,2016
2,268410,1,F,2016
3,271464,1,M,2016
4,274518,1,M,2016


## keep only students who occur in sem1 and sem6

In [9]:
# students in semester 1 - first occurence
idx_sem1 = (data.Scipper).isin(data[data.Semester == 1].Scipper)
data_sem1 = data[idx_sem1].drop('Semester',axis=1)
data_sem1 = data_sem1.sort_values(by ='Year')
data_sem1 = data_sem1.drop_duplicates(['Scipper'],keep='first')
data_sem1 = data_sem1.rename(columns = {'Year':'Year1'})
data_sem1.head()

,Scipper,Sex,Year1
1661,180284,M,2007
1691,180853,M,2007
1690,180094,M,2007
1689,181115,M,2007
1688,175576,M,2007


In [10]:
# students in semester 6 - last occurence
idx_sem6 = (data.Scipper).isin(data[data.Semester == 6].Scipper)
data_sem6 = data[idx_sem6].drop('Semester',axis=1)
data_sem6 = data_sem6.sort_values(by ='Year')
data_sem6 = data_sem6.drop_duplicates(['Scipper'],keep='last')
data_sem6 = data_sem6.rename(columns = {'Year':'Year6'})
data_sem6.head()

,Scipper,Sex,Year6
2383,171042,M,2008
2382,167439,M,2008
2350,161634,M,2008
2351,170451,M,2008
2352,170219,M,2008


In [11]:
data_sem16 = pd.merge(data_sem1,data_sem6,how='inner')
data_sem16.head()

,Scipper,Sex,Year1,Year6
0,180094,M,2007,2010
1,181115,M,2007,2010
2,181076,M,2007,2011
3,181298,M,2007,2010
4,178433,M,2007,2010


# compute stay time

In [12]:
data_sem16['Staytime'] = (data_sem16.Year6 - data_sem16.Year1)*12
data_sem16 = data_sem16.drop(['Year1','Year6'],axis=1)
data_sem16.head()

,Scipper,Sex,Staytime
0,180094,M,36
1,181115,M,36
2,181076,M,48
3,181298,M,36
4,178433,M,36


# partitonning the data + comparing average

In [13]:
data_grouped = data_sem16.groupby('Sex')
data_grouped['Staytime'].mean()

Sex
F    39.724138
M    41.771739
Name: Staytime, dtype: float64

# Statistical tests

In [14]:
import scipy.stats as stats

## Divide Populations

In [15]:
data_F = data_sem16[data_sem16.Sex == 'F']
data_M = data_sem16[data_sem16.Sex == 'M']

## 1-sample T-Test

In a 1-sample T-Test, the null hypothesis assumes nothing interesting is going on between the variables we are testing. In this case, it means that there is no difference between each of the sub-populations and the whole population.

In [16]:
stats.ttest_1samp(data_M.Staytime,data_sem16.Staytime.mean())

Ttest_1sampResult(statistic=0.32081543725818445, pvalue=0.74853286145726683)

A p-value of 0.7485 means we'd expect to see data as extreme as our sample due to chance about 74.85% of the time if the null hypothesis was true. In this case, the p-value is higher than our significance level α (equal to 1-conf.level or 0.05) so we should not reject the null hypothesis.
#### => The stay-time average of males is statically significant.

In [17]:
stats.ttest_1samp(data_F.Staytime,data_sem16.Staytime.mean())

Ttest_1sampResult(statistic=-1.5732943251612512, pvalue=0.12688368345278089)

A p-value of 0.1268 means we'd expect to see data as extreme as our sample due to chance about only 12.68% of the time if the null hypothesis was true. 
In this case, the p-value is low than our significance level so we should reject the null hypothesis.
#### The stay-time average of females is not statically significant, since the female population is different.

## 2-sample T-Test

In a 2-sample T-Test, the null hypothesis states that the groups are the same.

In [18]:
stats.ttest_ind(a= data_M.Staytime, b = data_F.Staytime, equal_var = False)

Ttest_indResult(statistic=1.5831651359439409, pvalue=0.12191236829650401)

The test yields a p-value of 0.1219, which means there is a 12.19% chance we would see sample data this far apart if the two groups tested are actually identical. The null hypothesis should be rejected.
#### => We conclude that the differance in the average of the stay-time between males and females is not statically significant.